Implementation of Sparse GPR

Necessary imports

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
import gpytorch
from gpytorch.models import ApproximateGP
from torch.distributions import Normal
import optuna
from optuna.trial import TrialState
import pickle
import pandas as pd
from sklearn.cluster import KMeans
import uncertainty_toolbox as uct

# define the device for the setting
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# check the computer name and set the path accordingly
if os.environ['COMPUTERNAME'] == 'FYNN':            # name of surface PC
    sys.path.append(r'C:\Users\Surface\Masterarbeit')
elif os.environ['COMPUTERNAME'] == 'FYNNS-PC':  # desktop name
    sys.path.append(r'C:\Users\test\Masterarbeit')
    
else:
    raise ValueError("Unbekannter Computername: " + os.environ['COMPUTERNAME'])

from utils.data_prep import load_tranform_and_split_data, set_seed
from utils.metrics import evaluate_intervals

Transformation Pipeline for Approximating GPR

In [ ]:
#load and transform the data, split it into training, validation, and test sets
# the split ratio is 60% training, 20% validation, and 20%
# return the feature names for later use
X_train, X_val, X_test, y_train, y_val, y_test, feature_names = load_tranform_and_split_data('C1_V01_delta_kan', split_ratio=(0.6, 0.2, 0.2))

# convert the data to PyTorch tensors
X_train_tensor = torch.from_numpy(X_train).float()
X_val_tensor = torch.from_numpy(X_val).float()
X_test_tensor = torch.from_numpy(X_test).float() 
# Add extra dimension for compatibility to the target tensors
y_train_tensor = torch.from_numpy(y_train).float() 
y_val_tensor = torch.from_numpy(y_val).float()
y_test_tensor = torch.from_numpy(y_test).float()

# Move tensors to GPU if available
if torch.cuda.is_available():
    X_train_tensor = X_train_tensor.cuda()
    X_val_tensor = X_val_tensor.cuda()
    X_test_tensor = X_test_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    y_val_tensor = y_val_tensor.cuda()
    y_test_tensor = y_test_tensor.cuda()

Stochastic Variational GP Regression Implementation

Natural Gradient Descent with Variational Models for better and faster convergence

In [ ]:
#create a TensorDataset and DataLoader for the training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

# define the GP model
class GPModel(ApproximateGP):
    """
    Stochastic Variational Gaussian Process model with learnable inducing points.
    
    Args:
        inducing_points (torch.Tensor): Initial locations of inducing points (shape: [num_inducing, input_dim])
        kernel (gpytorch.kernels.Kernel): Kernel function (e.g., RBF, RQ)
    
    Attributes:
        mean_module: Constant mean function initialized with training data mean
        covar_module: Kernel function for covariance
    """
    def __init__(self, inducing_points, kernel):
        variational_distribution = gpytorch.variational.NaturalVariationalDistribution(inducing_points.size(0)) #default value
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        ) #default value

        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean() # Constant mean function
        self.covar_module  = kernel
        self.mean_module.initialize(constant=y_train_tensor.mean().item())  # Initialize the mean with the mean of the target values of the training dataset

 #  define the forward method               
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

Create Optuna Study

In [ ]:
def objective(trial):
    """
    Optuna objective function for hyperparameter optimization of SVGP model.
    
    Optimizes learning rates (NGD, Adam) using validation NLL as metric.
    Early stopping prevents overfitting during hyperparameter search.
    
    Args:
        trial (optuna.Trial): Optuna trial object for suggesting hyperparameters
        
    Returns:
        float: Best validation loss (negative ELBO) achieved during training
        
    Note:
        Hyperparameter ranges and kernel choice are justified in Chapter X.Y
    """

    # Calculate the variance of the target values of the training dataset for initializing the likelihood noise
    y_var = y_train_tensor.var(unbiased=False).item()
    noise = 1e-2 * y_var # Initialize noise as 1% of target variance (prevents underestimation)

    # Suggest hyperparameters for optimization
    lr_ngd = trial.suggest_float('lr', 1e-3, 1e-1, log=True)
    lr_adam = trial.suggest_float('lr_adam', 1e-3, 1e-1, log=True)

    #Rational Quadratic Kernel
    rational_quadratic_kernel = gpytorch.kernels.RQKernel(ard_num_dims=X_train.shape[1], 
                                                          alpha_constraint=gpytorch.constraints.Interval(0.1, 10.0)) # Set ARD for all input dimensions
    rational_quadratic_kernel.lengthscale = torch.ones(X_train.shape[1]) # Initialize lengthscale to 1 for all dimensions
    rational_quadratic_kernel.outputscale = y_var # Initialize outputscale to the variance of the target values
    kernel = gpytorch.kernels.ScaleKernel(rational_quadratic_kernel) # Wrap RQ kernel in ScaleKernel to have outputscale parameter

    # Define the inducing points
    num_inducing_points = 1000

    # Use KMeans to select inducing points that represent the data distribution better
    kmeans = KMeans(n_clusters=num_inducing_points, random_state=42).fit(X_train)
    inducing_points = torch.from_numpy(kmeans.cluster_centers_).float()

    # Initialize the GP model and likelihood
    model = GPModel(inducing_points = inducing_points, kernel=kernel)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    likelihood.noise = noise

    print(f'Model: {model}')
    print(f'Mean: {model.mean_module.constant.item()}')
    print(f'Lengthscale: {model.covar_module.base_kernel.lengthscale}')
    print(f'Outputscale: {model.covar_module.outputscale}')
    print(f'Likelihood Noise: {likelihood.noise.item()}')
    # Move model and likelihood to GPU if available
    if torch.cuda.is_available():
        model = model.cuda()
        likelihood = likelihood.cuda()

    num_epochs = 100
    model.train()
    likelihood.train()
    # Use Natural Gradient Descent for the variational parameters and Adam for the hyperparameters of the kernel and likelihood
    variational_ngd_optimizer = gpytorch.optim.NGD(model.variational_parameters(), 
                                                   num_data= y_train_tensor.size(0), 
                                                   lr=lr_ngd)

    hyperparameter_optimizer = torch.optim.Adam([{'params': model.hyperparameters()},
                                                 {'params': likelihood.parameters()},
                                                 ],
                                                  lr=lr_adam)

    # VariationalELBO is used for training
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=len(train_loader.dataset))
    # Early stopping parameters
    best_val_loss = np.inf
    patience = 10
    epochs_no_improve = 0
    decimal_places = 3
    tolerance = 10 ** (-decimal_places) # Early stopping tolerance: 0.001 (prevents premature stopping on small fluctuations)

    # Training loop with early stopping
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        model.train()
        likelihood.train()
        for x_batch, y_batch in train_loader:
            # Zero gradients from previous iteration        
            variational_ngd_optimizer.zero_grad()
            hyperparameter_optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            loss.backward()
            variational_ngd_optimizer.step()
            hyperparameter_optimizer.step()
            epoch_loss += loss.item()
        # print every ten epochs
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader):.4f}')         # Print the average loss for the epoch

        # Validation step
        model.eval()
        likelihood.eval()
        with torch.no_grad():
            f_val_preds = model(X_val_tensor)
            val_loss = -mll(f_val_preds, y_val_tensor).item()  
            
            val_preds = likelihood(f_val_preds)     
            # Extract mean and standard deviation, Compute Negative Log-Likelihood (NLL) on validation set 
            # NLL = -log(p(y|x)) measures calibration quality of uncertainty estimates
            val_mean = val_preds.mean.cpu()
            val_std = val_preds.stddev.cpu()
            val_std = val_std.clamp_min(1e-6) # Minimum std to avoid log(0) in NLL computation
            nll_per_point = -Normal(val_mean, val_std).log_prob(y_val_tensor.cpu()).numpy()
            val_nll = nll_per_point.mean().item()
            
            r2_score_val = r2_score(y_val, val_mean)
            if (epoch + 1) % 10 == 0:
                print(f'Validation Loss: {val_loss} Validation NLL: {val_nll} R²: {r2_score_val:.3f}')
            # Report the intermediate value to Optuna
            trial.report(val_loss, step=epoch)

            # Handle pruning based on the intermediate value
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

            if abs(val_nll - best_val_loss) < tolerance:

                epochs_no_improve += 1
            else:
                epochs_no_improve = 0
                best_val_loss = val_loss
                # best_model_state = model.state_dict()
                # best_likelihood_state = likelihood.state_dict()   
            
            if epochs_no_improve >= patience:
                print(f'Early stopping at iteration {epoch + 1} with best validation loss: {best_val_loss:.3f}')
                # model.load_state_dict(best_model_state)
                # likelihood.load_state_dict(best_likelihood_state)

                break
    return best_val_loss

Execute Optuna Study

In [ ]:
# create a study object for Optuna
study = optuna.create_study(

    direction="minimize",
    sampler=optuna.samplers.TPESampler(),                       #TPE (Tree-structured Parzen Estimator) sampler by default
    pruner=optuna.pruners.MedianPruner(        
        n_startup_trials=5,                                    # Number of trials to run before pruning starts
        n_warmup_steps=5                                        # Number of warmup steps before pruning starts)
    )
)

# optimize the objective function with Optuna
# timeout=None means no time limit for the optimization, all trials will be run
study.optimize(objective, n_trials=50, timeout=None, n_jobs=1, show_progress_bar=True)

pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Make 10 Runs with different Random Seed to evaluate GPR

In [ ]:
# Lists to store results and predictions for each run
results_list = []
predictions_list = []
list_of_seeds = [42, 123, 777, 2024, 5250, 8888, 9876, 10001, 31415, 54321]
DE_prediction_path = r"C:\Users\test\Masterarbeit\models\Modelresults\GPR"
DE_result_path = r"C:\Users\test\OneDrive\Master Management und Engineering\Masterarbeit\Experimente\Evaluation\10 Runs\GPR"

for run, seed in enumerate(list_of_seeds):

    print(f"Run {run+1} with seed {seed}")
    set_seed(seed)
    # Calculate the variance of the target values of the training dataset for initializing the likelihood noise
    y_var = y_train_tensor.var(unbiased=False).item()
    noise = 1e-2 * y_var # Initialize noise as 1% of target variance (prevents underestimation)
    #Rational Quadratic Kernel and initialization of hyperparameters
    rational_quadratic_kernel = gpytorch.kernels.RQKernel(ard_num_dims=X_train.shape[1], 
                                                        alpha_constraint=gpytorch.constraints.Interval(0.1, 10.0)
                                                        )
    rational_quadratic_kernel.lengthscale = torch.ones(X_train.shape[1])
    rational_quadratic_kernel.outputscale = y_var
    kernel = gpytorch.kernels.ScaleKernel(rational_quadratic_kernel)
    #define the number of inducing points, increased to 2000 for better performance
    num_inducing_points = 2000
    # select inducing points with k-means
    kmeans = KMeans(n_clusters=num_inducing_points, random_state=42).fit(X_train)
    inducing_points = torch.from_numpy(kmeans.cluster_centers_).float()
    # create model and likelihood
    model = GPModel(inducing_points = inducing_points, kernel=kernel)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    # Initialize the likelihood noise
    likelihood.noise = noise
    print(f'Model: {model}')
    print(f'Mean: {model.mean_module.constant.item()}')
    print(f'Lengthscale: {model.covar_module.base_kernel.lengthscale}')
    print(f'Outputscale: {model.covar_module.outputscale}')
    print(f'Likelihood Noise: {likelihood.noise.item()}')
    if torch.cuda.is_available():
        model = model.cuda()
        likelihood = likelihood.cuda()

    num_epochs = 100
    # set the model and likelihood in training mode
    model.train()
    likelihood.train()
    # define the optimizers
    variational_ngd_optimizer = gpytorch.optim.NGD(model.variational_parameters(), 
                                                   num_data= y_train_tensor.size(0), 
                                                   lr=0.03)
    hyperparameter_optimizer = torch.optim.Adam([{'params': model.hyperparameters()},
                                                 {'params': likelihood.parameters()},
                                                 ], 
                                                 lr=0.04)

    # VariationalELBO is used for training
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=len(train_loader.dataset))

    # Early stopping parameters
    best_val_loss = np.inf
    patience = 10
    epochs_no_improve = 0
    decimal_places = 3
    tolerance = 10 ** (-decimal_places) # Early stopping tolerance: 0.001 (prevents premature stopping on small fluctuations)

    # Training loop with early stopping
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        model.train()
        likelihood.train()
        for x_batch, y_batch in train_loader:
            # Zero gradients from previous iteration        
            variational_ngd_optimizer.zero_grad()
            hyperparameter_optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            loss.backward()
            variational_ngd_optimizer.step()
            hyperparameter_optimizer.step()

            epoch_loss += loss.item()
        # print every ten epochs
        if (epoch + 1) % 10 == 0:
            print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss / len(train_loader):.4f}')         # Print the average loss for the epoch

        # Validation step
        model.eval()
        likelihood.eval()
        
        with torch.no_grad():
            f_val_preds = model(X_val_tensor)
            val_loss = -mll(f_val_preds, y_val_tensor).item()  
            
            # Extract mean and standard deviation, Compute Negative Log-Likelihood (NLL) on validation set 
            # NLL = -log(p(y|x)) measures calibration quality of uncertainty estimates
            val_preds = likelihood(f_val_preds)     
            val_mean = val_preds.mean.cpu()
            val_std = val_preds.stddev.cpu()
            val_std = val_std.clamp_min(1e-6) # Minimum std to avoid log(0) in NLL computation
            nll_per_point = -Normal(val_mean, val_std).log_prob(y_val_tensor.cpu()).numpy()
            val_nll = nll_per_point.mean().item()

            if (epoch + 1) % 10 == 0:
                print(f'Validation Loss: {val_loss} Validation NLL: {val_nll}')
            
            # Early stopping check
            if abs(val_nll - best_val_loss) < tolerance:

                epochs_no_improve += 1
            else:
                epochs_no_improve = 0
                best_val_loss = val_loss
                best_model_state = model.state_dict()
                best_likelihood_state = likelihood.state_dict()   
            
            if epochs_no_improve >= patience:
                print(f'Early stopping at iteration {epoch + 1} with best validation loss: {best_val_loss:.3f}')
                model.load_state_dict(best_model_state)
                likelihood.load_state_dict(best_likelihood_state)

                break


    # Evaluation on the test set            
    model.eval()
    likelihood.eval()
    means = []
    stddevs = []
    with torch.no_grad():
        # Make predictions on the test set
        preds = likelihood(model(X_test_tensor))      
        # Mean:
        means.append(preds.mean.cpu())        
        # Standard Deviation
        stddevs.append(preds.stddev.cpu())

    GPR_means = torch.cat(means)
    GPR_stddevs = torch.cat(stddevs)
    #convert to numpy arrays
    GPR_means = GPR_means.numpy()
    GPR_stddevs = GPR_stddevs.numpy()

    # Calculate and print all metrics inclunding RMSE, MAE, R²-Score, NLL, CRPS
    pnn_metrics = uct.metrics.get_all_metrics(GPR_means, GPR_stddevs, y_test)
    print(pnn_metrics)

    # use own function to calculate coverage and MPIW
    ev_intervals = evaluate_intervals(GPR_means, GPR_stddevs, y_test, coverage=0.95)
    print(f'coverage: {ev_intervals["coverage"]}, MPIW: {ev_intervals["MPIW"]}')

    # Store predictions and results for this run
    predictions_per_run = {
                            'mean_prediction': GPR_means,
                            'std_prediction': GPR_stddevs,
    }

    results_per_run = {
                        'RMSE': pnn_metrics['accuracy']['rmse'],
                        'MAE': pnn_metrics['accuracy']['mae'],
                        'R2': pnn_metrics['accuracy']['r2'], 
                        'Correlation' : pnn_metrics['accuracy']['corr'],
                        'NLL': pnn_metrics['scoring_rule']['nll'],
                        'CRPS': pnn_metrics['scoring_rule']['crps'],
                        'coverage': ev_intervals["coverage"],
                        'MPIW': ev_intervals["MPIW"],
                        }

    predictions_list.append(predictions_per_run)
    results_list.append(results_per_run)
    
#save the predictions 
with open(os.path.join(DE_prediction_path, "GPR_predictions_list.pkl"), "wb") as f:
    pickle.dump(predictions_list, f)

#save the results in an excel file
results_df = pd.DataFrame(results_list)
results_df.to_excel(os.path.join(DE_result_path, "GPR_results.xlsx"), index=False)

Evaluation of the 10 runs

In [ ]:
DE_prediction_path = r"C:\Users\test\Masterarbeit\models\Modelresults\GPR"
DE_result_path = r"C:\Users\test\OneDrive\Master Management und Engineering\Masterarbeit\Experimente\Evaluation\10 Runs\GPR"
with open(os.path.join(DE_prediction_path, "GPR_predictions_list.pkl"), "rb") as f:
    predictions_list = pickle.load(f)

# Lists to store mean and std predictions across runs
mean_list = []
std_list = []

for id, run in enumerate(predictions_list):
    # extract mean and std predictions
    mean = run['mean_prediction']
    std = run['std_prediction']
    
    # append to lists
    mean_list.append(mean)
    std_list.append(std)
    
    # calibration Curve with UCT
    uct.viz.plot_calibration(mean, std, y_test)
    plt.savefig(os.path.join(DE_result_path, f"calibration_run_{id+1}.svg"), format ='svg')
    plt.close()

    # adversarial group calibration
    uct.viz.plot_adversarial_group_calibration(mean, std, y_test)
    plt.savefig(os.path.join(DE_result_path, f"adversarial_group_calibration_run_{id+1}.svg"), format ='svg')
    plt.close()

# predictions_list enthält pro Run ein Array mit 10403 Werten
mean_matrix = np.array(mean_list)  # Shape: (n_runs, 10403)
std_matrix = np.array(std_list)    # Shape: (n_runs, 10403)

# Mittelwert und Std für jeden Datenpunkt über alle Runs
mean_per_datapoint = np.mean(mean_matrix, axis=0)  # Shape: (10403,)
std_per_datapoint = np.mean(std_matrix, axis=0)    # Shape: (10403,)

# calibration Curve with UCT
uct.viz.plot_calibration(mean_per_datapoint, std_per_datapoint, y_test)
plt.savefig(os.path.join(DE_result_path, "calibration_run_mean.svg"), format ='svg')
plt.close()

# adversarial group calibration
uct.viz.plot_adversarial_group_calibration(mean_per_datapoint, std_per_datapoint, y_test)
plt.savefig(os.path.join(DE_result_path, "adversarial_group_calibration_run_mean.svg"), format ='svg')
plt.close()